In [1]:
%matplotlib inline
from IPython.display import Image, HTML
import json
import datetime
import ast
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy import stats
from sklearn.ensemble import GradientBoostingClassifier, GradientBoostingRegressor
from sklearn.dummy import DummyClassifier, DummyRegressor
from sklearn.model_selection import train_test_split
from wordcloud import WordCloud, STOPWORDS
import plotly
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
import warnings
warnings.filterwarnings('ignore')
plotly.tools.set_credentials_file(username='rounakbanik', api_key='xTLaHBy9MVv5szF4Pwan')
plt.rcParams['font.sans-serif']=['SimHei'] # 用来正常显示中文标签 
plt.rcParams['axes.unicode_minus']=False # 用来正常显示正负号
#sns.set_style('whitegrid')
#sns.set(font_scale=1.25)
pd.set_option('display.max_colwidth', 50)

In [2]:
# 导入数据
df = pd.read_excel('/Users/chandler/Desktop/618_bhsm/jihe/2018.xlsx')
df.head()

,推送类型,渠道,发送日期,数据日期,活动类型,活动编号,活动名称,活动归属编码,活动归属名称,计划发送会员数,...,PV,加入购物车UV,加入购物车转化率,提交订单数,提交订单金额,下单会员数,买家数,成功支付订单数,付款金额,客单价
0,精推,站内信,2018-02-05,2018-02-07,-,C0004_2018/02/05,C0004_2018/02/05,-,-,1953476.0,...,2136.0,0.0,0.0,153.0,137519.09,76.0,72.0,106.0,74440.94,1033.901944
1,精推,站内信,2018-03-05,2018-03-07,-,C0004_2018/03/05,C0004_2018/03/05,-,-,2227871.0,...,1790.0,0.0,0.0,69.0,85963.94,46.0,44.0,55.0,50043.27,1137.347045
2,精推,站内信,2018-02-05,2018-02-07,-,C0005_2018/02/05,C0005_2018/02/05,-,-,369542.0,...,421.0,0.0,0.0,26.0,20534.68,10.0,10.0,26.0,19909.71,1990.971000
3,精推,站内信,2018-03-05,2018-03-07,-,C0005_2018/03/05,C0005_2018/03/05,-,-,747191.0,...,721.0,0.0,0.0,32.0,114552.62,16.0,15.0,24.0,64011.76,4267.450667
4,精推,站内信,2018-01-01,2018-01-03,-,C0006_2018/01/01,C0006_2018/01/01,-,-,1875.0,...,65.0,0.0,0.0,1.0,11.90,1.0,1.0,1.0,1.21,1.210000


In [3]:
df.columns

Index(['推送类型', '渠道', '发送日期', '数据日期', '活动类型', '活动编号', '活动名称', '活动归属编码',
       '活动归属名称', '计划发送会员数', '发送成功会员数', '发送成功率', 'A-打开率', 'B-点击率', '人均下单数',
       '下单会员数1', '买家数1', '成功支付订单数1', '付款金额1', '客单价1', 'C-转化率1', '下单会员数2',
       '买家数2', '成功支付订单数2', '付款金额2', '客单价2', 'D-目标转化率2', '下单会员数3', '买家数3',
       '成功支付订单数3', '付款金额3', '客单价3', 'C-转化率3', '下单会员数4', '买家数4', '成功支付订单数4',
       '付款金额4', '客单价4', 'D-目标转化率4', '反馈会员数', '打开会员数', '点击会员数', 'UV', 'PV',
       '加入购物车UV', '加入购物车转化率', '提交订单数', '提交订单金额', '下单会员数', '买家数', '成功支付订单数',
       '付款金额', '客单价'],
      dtype='object')

In [4]:
df = df[['发送日期', '推送类型', '渠道', '活动归属名称', '计划发送会员数', '买家数2', '买家数4', '打开会员数', 'UV']]

In [5]:
df.head(20)

,发送日期,推送类型,渠道,活动归属名称,计划发送会员数,买家数2,买家数4,打开会员数,UV
0,2018-02-05,精推,站内信,-,1953476.0,0.0,0.0,548.0,173.0
1,2018-03-05,精推,站内信,-,2227871.0,0.0,0.0,523.0,139.0
2,2018-02-05,精推,站内信,-,369542.0,0.0,0.0,96.0,33.0
3,2018-03-05,精推,站内信,-,747191.0,0.0,0.0,113.0,46.0
4,2018-01-01,精推,站内信,-,1875.0,0.0,0.0,7.0,5.0
5,2018-01-02,精推,站内信,-,2132.0,0.0,0.0,4.0,3.0
6,2018-01-03,精推,站内信,-,1844.0,0.0,0.0,10.0,6.0
7,2018-01-04,精推,站内信,-,2257.0,0.0,0.0,10.0,6.0
8,2018-01-05,精推,站内信,-,1708.0,0.0,0.0,10.0,3.0
9,2018-01-06,精推,站内信,-,1930.0,0.0,0.0,5.0,4.0


In [6]:
act_belongto_name = [
    '营销管理总部-百货事业部', '营销管理总部-冰洗事业部', '营销管理总部超市公司',
    '营销管理总部-厨卫事业部', '营销管理总部-电脑事业部', '营销管理总部-黑电事业部',
    '营销管理总部-红孩子公司', '营销管理总部-空调事业部', '营销管理总部-生活电器事业部',
    '营销管理总部-数码事业部', '营销管理总部-通讯公司', '运营总部-海外购事业部',
    '运营总部-会员管理中心'
]

# 循环遍历df的所有索引
for i in df.index:
    # 判断如果是短信渠道就要删除'买家数4', '打开会员数', 'UV'这三个列的所有数据并赋值为0
    if df['渠道'].iloc[i] == '短信':
        df['打开会员数'].iat[i] = 0
        df['UV'].iat[i] = 0
        df['买家数4'].iat[i] = 0
    # 否则就要删除'计划发送会员数', '买家数2'这二个列的所有数据并赋值为0
    if df['渠道'].iloc[i] != '短信':
        df['计划发送会员数'].iat[i] = 0
        df['买家数2'].iat[i] = 0

# 整个df只要活动归属名称在act_belongto_name这个列表中的所有数据
df = df[df['活动归属名称'].isin(act_belongto_name)]
# 对处理后的dataframe重新索引（删除一些数据，索引可能出错，比如出现0，1，2，3，12，13这样的断点）

In [7]:
df.head(20)

,发送日期,推送类型,渠道,活动归属名称,计划发送会员数,买家数2,买家数4,打开会员数,UV
993,2018-01-02,精推,站内信,营销管理总部-黑电事业部,0.0,0.0,18.0,178.0,46.0
994,2018-01-01,精推,站内信,营销管理总部-通讯公司,0.0,0.0,102.0,370.0,104.0
995,2018-01-01,精推,站内信,营销管理总部-通讯公司,0.0,0.0,19.0,63.0,16.0
996,2018-01-02,精推,站内信,营销管理总部-生活电器事业部,0.0,0.0,95.0,689.0,107.0
997,2018-01-02,精推,站内信,营销管理总部-数码事业部,0.0,0.0,10.0,278.0,49.0
998,2018-01-02,精推,站内信,营销管理总部-数码事业部,0.0,0.0,24.0,484.0,53.0
999,2018-01-02,精推,站内信,营销管理总部-黑电事业部,0.0,0.0,44.0,412.0,117.0
1000,2018-01-02,精推,站内信,营销管理总部-黑电事业部,0.0,0.0,7.0,30.0,11.0
1001,2018-01-03,精推,站内信,营销管理总部-黑电事业部,0.0,0.0,3.0,35.0,6.0
1002,2018-01-03,精推,站内信,营销管理总部-黑电事业部,0.0,0.0,9.0,85.0,20.0


In [8]:
df = df.sort_values(by=['发送日期','活动归属名称'])

In [9]:
df.head(20)

,发送日期,推送类型,渠道,活动归属名称,计划发送会员数,买家数2,买家数4,打开会员数,UV
10664,2018-01-01,精推,短信,营销管理总部-厨卫事业部,637.0,83.0,0.0,0.0,0.0
10665,2018-01-01,精推,短信,营销管理总部-厨卫事业部,2190.0,193.0,0.0,0.0,0.0
10666,2018-01-01,精推,短信,营销管理总部-厨卫事业部,5384.0,205.0,0.0,0.0,0.0
10675,2018-01-01,精推,短信,营销管理总部-厨卫事业部,327.0,55.0,0.0,0.0,0.0
10670,2018-01-01,精推,短信,营销管理总部-电脑事业部,1155.0,68.0,0.0,0.0,0.0
10673,2018-01-01,精推,短信,营销管理总部-电脑事业部,5421.0,139.0,0.0,0.0,0.0
10674,2018-01-01,精推,短信,营销管理总部-电脑事业部,1861.0,64.0,0.0,0.0,0.0
1913,2018-01-01,精推,APP PUSH,营销管理总部-百货事业部,0.0,0.0,68.0,452.0,452.0
10667,2018-01-01,精推,短信,营销管理总部-百货事业部,1604.0,37.0,0.0,0.0,0.0
15955,2018-01-01,精推,消息中心,营销管理总部-百货事业部,0.0,0.0,176.0,793.0,452.0


In [10]:
df['营销数量'] = df['打开会员数'] + df['计划发送会员数']
df['贡献买家数'] = df['买家数2'] + df['买家数4']
df['转化率'] = df['贡献买家数'] / df['营销数量']

In [11]:
df.head(20)

,发送日期,推送类型,渠道,活动归属名称,计划发送会员数,买家数2,买家数4,打开会员数,UV,营销数量,贡献买家数,转化率
10664,2018-01-01,精推,短信,营销管理总部-厨卫事业部,637.0,83.0,0.0,0.0,0.0,637.0,83.0,0.130298
10665,2018-01-01,精推,短信,营销管理总部-厨卫事业部,2190.0,193.0,0.0,0.0,0.0,2190.0,193.0,0.088128
10666,2018-01-01,精推,短信,营销管理总部-厨卫事业部,5384.0,205.0,0.0,0.0,0.0,5384.0,205.0,0.038076
10675,2018-01-01,精推,短信,营销管理总部-厨卫事业部,327.0,55.0,0.0,0.0,0.0,327.0,55.0,0.168196
10670,2018-01-01,精推,短信,营销管理总部-电脑事业部,1155.0,68.0,0.0,0.0,0.0,1155.0,68.0,0.058874
10673,2018-01-01,精推,短信,营销管理总部-电脑事业部,5421.0,139.0,0.0,0.0,0.0,5421.0,139.0,0.025641
10674,2018-01-01,精推,短信,营销管理总部-电脑事业部,1861.0,64.0,0.0,0.0,0.0,1861.0,64.0,0.034390
1913,2018-01-01,精推,APP PUSH,营销管理总部-百货事业部,0.0,0.0,68.0,452.0,452.0,452.0,68.0,0.150442
10667,2018-01-01,精推,短信,营销管理总部-百货事业部,1604.0,37.0,0.0,0.0,0.0,1604.0,37.0,0.023067
15955,2018-01-01,精推,消息中心,营销管理总部-百货事业部,0.0,0.0,176.0,793.0,452.0,793.0,176.0,0.221942


In [12]:
df['发送日期'] = pd.to_datetime(df['发送日期'], format='%Y/%m')

In [13]:
df.head()

,发送日期,推送类型,渠道,活动归属名称,计划发送会员数,买家数2,买家数4,打开会员数,UV,营销数量,贡献买家数,转化率
10664,2018-01-01,精推,短信,营销管理总部-厨卫事业部,637.0,83.0,0.0,0.0,0.0,637.0,83.0,0.130298
10665,2018-01-01,精推,短信,营销管理总部-厨卫事业部,2190.0,193.0,0.0,0.0,0.0,2190.0,193.0,0.088128
10666,2018-01-01,精推,短信,营销管理总部-厨卫事业部,5384.0,205.0,0.0,0.0,0.0,5384.0,205.0,0.038076
10675,2018-01-01,精推,短信,营销管理总部-厨卫事业部,327.0,55.0,0.0,0.0,0.0,327.0,55.0,0.168196
10670,2018-01-01,精推,短信,营销管理总部-电脑事业部,1155.0,68.0,0.0,0.0,0.0,1155.0,68.0,0.058874


In [14]:
df = df.reset_index(drop=True)

In [15]:
df.shape

(15691, 12)

In [25]:
time_ = pd.to_datetime(['2018/03/30'])
# 循环遍历df的所有索引
df['季度'] = 't'
for i in df.index:
    # 判断如果是短信渠道就要删除'买家数4', '打开会员数', 'UV'这三个列的所有数据并赋值为0
    if df['发送日期'].iat[i] <= time_:
        df['季度'].iat[i] = '第一季度'
    if df['发送日期'].iat[i] >= time_:
        df['季度'].iat[i] = '第二季度'


In [42]:
df.head(50)

,发送日期,推送类型,渠道,活动归属名称,计划发送会员数,买家数2,买家数4,打开会员数,UV,营销数量,贡献买家数,转化率,季度
0,2018-01-01,精推,短信,营销管理总部-厨卫事业部,637.0,83.0,0.0,0.0,0.0,637.0,83.0,0.130298,第一季度
1,2018-01-01,精推,短信,营销管理总部-厨卫事业部,2190.0,193.0,0.0,0.0,0.0,2190.0,193.0,0.088128,第一季度
2,2018-01-01,精推,短信,营销管理总部-厨卫事业部,5384.0,205.0,0.0,0.0,0.0,5384.0,205.0,0.038076,第一季度
3,2018-01-01,精推,短信,营销管理总部-厨卫事业部,327.0,55.0,0.0,0.0,0.0,327.0,55.0,0.168196,第一季度
4,2018-01-01,精推,短信,营销管理总部-电脑事业部,1155.0,68.0,0.0,0.0,0.0,1155.0,68.0,0.058874,第一季度
5,2018-01-01,精推,短信,营销管理总部-电脑事业部,5421.0,139.0,0.0,0.0,0.0,5421.0,139.0,0.025641,第一季度
6,2018-01-01,精推,短信,营销管理总部-电脑事业部,1861.0,64.0,0.0,0.0,0.0,1861.0,64.0,0.034390,第一季度
7,2018-01-01,精推,APP PUSH,营销管理总部-百货事业部,0.0,0.0,68.0,452.0,452.0,452.0,68.0,0.150442,第一季度
8,2018-01-01,精推,短信,营销管理总部-百货事业部,1604.0,37.0,0.0,0.0,0.0,1604.0,37.0,0.023067,第一季度
9,2018-01-01,精推,消息中心,营销管理总部-百货事业部,0.0,0.0,176.0,793.0,452.0,793.0,176.0,0.221942,第一季度


In [46]:
import os
df.to_excel('suncheng.xlsx')